In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
G

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-06-13 03:41:47--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-06-13 03:41:47 (9.88 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [6]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://.czxhf6qdffub.us-east-1.rds.amazonaws.com:5432/"
config = {"user":"postgres", 
          "password": "", 
          "driver":"org.postgresql.Driver"}

In [7]:
df = spark.read.jdbc(url=jdbc_url, table='vine_table', properties=config)

In [8]:
df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R9CO86UUJCAW5|          3|            0|          0|   N|                Y|
|R3PR8X6QGVJ8B1|          5|            0|          0|   N|                Y|
|R39BO2819ABUPF|          4|            0|          0|   N|                Y|
|R3ADL7V6EGGEEP|          4|            0|          0|   N|                Y|
|R1OXYPBPLVRMI5|          5|            0|          0|   N|                Y|
|R1WYM8Z5ATQ98O|          3|            0|          0|   N|                Y|
|R3LCIANTN1H9EC|          4|            1|          1|   N|                Y|
|R3U2M23N1P0KQ6|          5|            0|          0|   N|                Y|
|R29MB6N7HB6NZI|          1|            2|          2|   N|                Y|
| RGEQ6DGRG7DQG|          5|            0|          0|   N|     

In [9]:
df_g20 = df.filter(df.total_votes > 20)

In [10]:
df_g20.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R11IBSD5E6HPSD|          1|           29|         31|   N|                Y|
|R2KVWAYBPWK1OV|          5|           20|         22|   N|                N|
|R1FLOE9E4ODIGR|          5|           34|         38|   N|                Y|
| R6XTEZCSCUJ4J|          5|           20|         23|   N|                Y|
|R2WVV9LQKIVMOX|          5|           23|         23|   N|                N|
| RYIU6GIB4TYT2|          5|           30|         30|   N|                N|
| RNXGKYVCJDDL7|          5|           28|         28|   N|                Y|
| RK52L3B5733BH|          5|           22|         22|   N|                N|
|R213PW7RSI6Z7N|          5|           48|         50|   N|                Y|
| RUBBVNQCSYCDF|          5|           23|         24|   N|     

In [11]:
df_g20_h50 = df_g20.filter( (df_g20.helpful_votes/df_g20.total_votes) >= 0.5)

In [12]:
df_g20_h50.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R11IBSD5E6HPSD|          1|           29|         31|   N|                Y|
|R2KVWAYBPWK1OV|          5|           20|         22|   N|                N|
|R1FLOE9E4ODIGR|          5|           34|         38|   N|                Y|
| R6XTEZCSCUJ4J|          5|           20|         23|   N|                Y|
|R2WVV9LQKIVMOX|          5|           23|         23|   N|                N|
| RYIU6GIB4TYT2|          5|           30|         30|   N|                N|
| RNXGKYVCJDDL7|          5|           28|         28|   N|                Y|
| RK52L3B5733BH|          5|           22|         22|   N|                N|
|R213PW7RSI6Z7N|          5|           48|         50|   N|                Y|
| RUBBVNQCSYCDF|          5|           23|         24|   N|     

In [13]:
df_g20_h50_revYes = df_g20_h50.filter(df_g20_h50.vine=='Y')

In [14]:
df_g20_h50_revYes.show(25)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RPJ7C4HP3BHXN|          5|           25|         29|   Y|                N|
|R3SB08XK0M7993|          4|           44|         49|   Y|                N|
|R26X1DACMCN3YF|          3|           16|         21|   Y|                N|
|R17HKKDE9ZRVOO|          4|           31|         35|   Y|                N|
|R1R7GNSCMO8U7I|          4|           24|         28|   Y|                N|
| RD006IWFCUA5A|          5|          122|        133|   Y|                N|
|R2T83WIPG3IX5D|          4|           72|         76|   Y|                N|
| REOQCBUVUPXP1|          4|           23|         27|   Y|                N|
|R2336R80ZEOKTX|          5|           63|         63|   Y|                N|
|R2J7WSJ8EX3D5S|          5|           30|         32|   Y|     

In [15]:
df_g20_h50_revNo = df_g20_h50.filter(df_g20_h50.vine=='N')

In [16]:
df_g20_h50_revNo.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R11IBSD5E6HPSD|          1|           29|         31|   N|                Y|
|R2KVWAYBPWK1OV|          5|           20|         22|   N|                N|
|R1FLOE9E4ODIGR|          5|           34|         38|   N|                Y|
| R6XTEZCSCUJ4J|          5|           20|         23|   N|                Y|
|R2WVV9LQKIVMOX|          5|           23|         23|   N|                N|
| RYIU6GIB4TYT2|          5|           30|         30|   N|                N|
| RNXGKYVCJDDL7|          5|           28|         28|   N|                Y|
| RK52L3B5733BH|          5|           22|         22|   N|                N|
|R213PW7RSI6Z7N|          5|           48|         50|   N|                Y|
| RUBBVNQCSYCDF|          5|           23|         24|   N|     

In [17]:
total_rev_paid = df_g20_h50_revYes.select('review_id').count()
total_rev_paid

21

In [18]:
total_5star_reviews_paid = df_g20_h50_revYes.filter(df_g20_h50_revYes.star_rating == 5).count()
total_5star_reviews_paid

10

In [19]:
percentage_5star_reviews_paid  = total_5star_reviews_paid/total_rev_paid*100
percentage_5star_reviews_paid

47.61904761904761

In [38]:
total_rev_unpaid = df_g20_h50_revNo.select('review_id').count()
total_rev_unpaid

6309

In [21]:

total_5star_reviews_unpaid = df_g20_h50_revNo.filter(df_g20_h50_revNo.star_rating == 5).count()
total_5star_reviews_unpaid

3266

In [22]:
percentage_5star_reviews_unpaid  = total_5star_reviews_unpaid/total_rev_unpaid*100
percentage_5star_reviews_unpaid

51.7673165319385

In [47]:
# from pyspark.sql.functions import agg_funcs
# df_g20_h50_revYes.select('star_rating').max()
df_g20_h50_revYes.agg({'star_rating':'mean'}).show()

+-----------------+
| avg(star_rating)|
+-----------------+
|4.285714285714286|
+-----------------+



In [43]:
df_g20_h50_revNo.agg({'star_rating':'mean'}).show()

+-----------------+
| avg(star_rating)|
+-----------------+
|3.805832937074021|
+-----------------+



In [49]:
df_g20_h50_revYes.describe().show()

+-------+-------------+------------------+------------------+------------------+----+-----------------+
|summary|    review_id|       star_rating|     helpful_votes|       total_votes|vine|verified_purchase|
+-------+-------------+------------------+------------------+------------------+----+-----------------+
|  count|           21|                21|                21|                21|  21|               21|
|   mean|         null| 4.285714285714286| 41.42857142857143| 44.38095238095238|null|             null|
| stddev|         null|0.9561828874675149|28.573714194293025|29.503349285252664|null|             null|
|    min|R140DK42LYQWL|                 1|                16|                21|   Y|                N|
|    max|RY1C7QLDFKZPG|                 5|               122|               133|   Y|                N|
+-------+-------------+------------------+------------------+------------------+----+-----------------+



In [50]:
df_g20_h50_revNo.describe().show()

+-------+--------------+------------------+-----------------+-----------------+----+-----------------+
|summary|     review_id|       star_rating|    helpful_votes|      total_votes|vine|verified_purchase|
+-------+--------------+------------------+-----------------+-----------------+----+-----------------+
|  count|          6309|              6309|             6309|             6309|6309|             6309|
|   mean|          null| 3.805832937074021|56.39911237914091|60.27880805198922|null|             null|
| stddev|          null|1.5124271390614188|76.06523273759865|78.60091657443071|null|             null|
|    min|R10090Y4VX54G9|                 1|               11|               21|   N|                N|
|    max| RZXGS4POIH67M|                 5|             1992|             2030|   N|                Y|
+-------+--------------+------------------+-----------------+-----------------+----+-----------------+

